In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )


a = datetime.now()-timedelta(days=1)
a = a.strftime('%Y.%m.%d')
b = datetime.now()-timedelta(days=13)
b = b.strftime('%Y.%m.%d')
print(b, a)

2021.04.23 2021.05.05


In [3]:
print("=" *80)
print("         BOBE 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\crawler-bot\\COMMUNITY\\{b}-{a}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         BOBE 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+'bobe.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
# keyword2 = []   # 크롤링 키워드
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
url2 = []       # 게시물 url
time2 = []      # 게시물 작성 시간
view_cnt2 = []  # 게시물 뷰 카운트
like_cnt2 = []  # 추천/공감 카운트
text2 = []      # 게시물 내용
com_cnt2 = []   # 댓글 수
com_text2 = []  # 댓글 user_id & text

In [6]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
# 1. 보배드림 크롤링
query_url= ('https://www.bobaedream.co.kr/list?code=politic')

driver.get(query_url)
driver.implicitly_wait(5)

In [8]:
domain1 = "보배드림"
print("==========",domain1,"크롤링 진행 중 ==========")

========== 보배드림 크롤링 진행 중 ==========


In [9]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
item_box = soup.find(name='table', attrs={'id':'boardlist'}).find('tbody')
item_box

<tbody>
<tr class="best"><td class="c"><img alt="1rank" class="imgmargin" src="//image.bobaedream.co.kr/renew/images/bobae_renew/num01.gif"/></td><td><a class="bsubject" href="/view?code=politic&amp;No=444716&amp;rtn=%2Flist%3Fcode%3Dpolitic">배신자 레전드.jpg</a><a href="javascript:window.open('/board/bulletin/comment_list.php?tb=uni_cmt_2104&amp;code=politic&amp;No=444716','','resizable=yes,scrollbars=yes,width=760,height=590');void(0);" style="padding-left:4px;"><span class="Comment">(<strong class="totreply">12</strong>)</span></a></td><td class="author02"><span class="author_level" onclick="javascript:submenu_level('b3BocXBvcGhyMm9waHFtb3BocWRvcGhxbW9waHFl');"><img alt="레벨 원사 3" class="level" src="//image.bobaedream.co.kr/board/level/army_9.gif"/></span><span class="author" onclick="javascript:submenu_show('b3BocXBvcGhyMm9waHFtb3BocWRvcGhxbW9waHFl', '');" style="cursor:pointer;">달려누냐</span></td><td class="date">04/30</td><td class="recomm"><font style="color:#ff7234;font-weight:bold;">3

In [10]:
page_num = 0        # 현재 페이지 넘버
while_code = 0

while True :
    
    if while_code == 1:
        break

    target_page = f"https://www.bobaedream.co.kr/list?code=politic&s_cate=&maker_no=&model_no=&or_gu=10&or_se=desc&s_selday=&pagescale=30&info3=&noticeShow=&s_select=&s_key=&level_no=&vdate=&type=list&page={page_num}"
    driver.get(target_page)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    item_box = soup.find(name='table', attrs={'id':'boardlist'}).find('tbody')

    for i in item_box :
        
        if len(i) > 1 :
            
            try :                
                # best 글 및 공지글 제외
                best_cut =  i['class'][0] == 'best'
                print("best 글입니다. 크롤링 대상에서 제외합니다.")
                
            except :
                target_url = "https://www.bobaedream.co.kr"+i.find(name='a', attrs={'class':'bsubject'})['href']
                driver.get(target_url)
                time.sleep(1)    
                post_html = driver.page_source
                post_soup = BeautifulSoup(post_html, 'html.parser')
                post_box = post_soup.find(name='div', attrs={'class':'docuArea02'})       
                
                
                target_day = b
                post_day = post_box.find(name='span', attrs={'class':'countGroup'}).text.split("|")[-1][:11].strip()
                print("scout :", post_day, target_day, post_day >= target_day)
                if post_day >= target_day :
                    
                    target_url = "https://www.bobaedream.co.kr"+i.find(name='a', attrs={'class':'bsubject'})['href']
                    print("target_url :", target_url)
                    driver.get(target_url)
                    time.sleep(1)
                    print("domain :", domain1)
#                     print("keyword :", keyword)

                    post_html = driver.page_source
                    post_soup = BeautifulSoup(post_html, 'html.parser')
                    post_box = post_soup.find(name='div', attrs={'class':'docuArea02'})

                    # title
                    title1 = post_box.find(name='strong', attrs={'itemprop':'name'}).text.strip().split("\n")[0]     
                    print("title :", title1)

                    # user_id
                    user1 = post_box.find(name='a', attrs={'class':'nickName'}).text.strip()            
                    print("user_id :", user1)

                    # time
                    time1 = post_box.find(name='span', attrs={'class':'countGroup'}).text.split("|")[-1][:11]
                    print("작성 시간 :", time1)

                    # view
                    view_cnt1 = post_box.find(name='span', attrs={'class':'countGroup'}).find_all(name='em', attrs={'class':'txtType'})[0].text.strip()
                    print("조회수 :", view_cnt1)

                    # like
                    like = post_box.find(name='span', attrs={'class':'countGroup'}).find_all(name='em', attrs={'class':'txtType'})[1].text.strip()
                    dislike = post_box.find(name='span', attrs={'class':'countGroup'}).find_all(name='em', attrs={'class':'txtType'})[2].text.strip()
                    like_cnt1 = like + " & " + dislike
                    print("추천/반대 수 :", like_cnt1)

                    # text
                    text1 = post_box.find(name='div', attrs={'class':'bodyCont'}).text.strip().replace("\n","")
                    print('게시물 내용 :', text1) 

                    # com_cnt
                    com_cnt1 = post_box.find(name='span', attrs={'class':'comm2'}).text.replace("(","").replace(")","")
                    print("댓글 수 :", com_cnt1)

                    try :

                        # com_text
                        com_box = post_box.find(name='ul', attrs={'class':'basiclist'})
                        com_list = []

                        for j in com_box :

                            if len(j) > 1 :  

                                # comment_user_id
                                com_user = j.find(name='span', attrs={'class':'name'}).text.strip()
                                com_text = j.find("dd").text.strip().replace("\n","")
                                com_list.append(com_user + " : " + com_text)
                    except :
                        com_list = []


                    print('게시물 댓글 :', com_list)

                    domain2.append(domain1)
#                     keyword2.append(keyword)
                    title2.append(title1)
                    user2.append(user1)
                    url2.append(target_url)
                    time2.append(time1)
                    view_cnt2.append(view_cnt1)
                    like_cnt2.append(like_cnt1)
                    text2.append(text1)
                    com_cnt2.append(com_cnt1)
                    com_text2.append(com_list)

                    print("\n")

                else :
                    print("크롤링을 완료했습니다.")
                    while_code = 1
                    break

    page_num += 1


scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445755&bm=1
domain : 보배드림
title : 민변 “文정부, 민생 외면하고 檢개혁 몰두… 그마저도 낙제점”[2]
user_id : 미국산앵무새
작성 시간 :  2021.05.06
조회수 : 25
추천/반대 수 : 4 & 0
게시물 내용 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=022&aid=0003578543세계일보민변 “文정부, 민생 외면하고 檢개혁 몰두… 그마저도 낙제점”  민변, 국정과제 6대 분야 평가檢총장 해임 과정서 과잉 정쟁LH 사태로 공정성 강조 ‘빈말로’박근혜 때보다 표현 자유는 신장 민주사회를 위한 변호사모임이 문재인정부 4년을 맞아 발간한 국정과제 평가 보고서에서 낙제점을 매겼다. 검찰개혁에 집중하느라 사회·경제·민생개혁은 소홀했고 검찰개혁조차 제대로 못했다는 이유에서다.  민변조차 문정부 낙제점 줬네요 ㅋㅋ근데 대통령이 직접 국민 고소하는데 표현의 자유라니?아....그거네요. 문빠들의 문자폭탄 양념질을 표현의 자유라고 하나 봅니다.
댓글 수 : 2
게시물 댓글 : ['모드칩 : 검찰 개혁이 결국 알고보니 퇴임 후 대통령의 안전확보였음을 국민이 알아차려 버렸다.', '친중매국노대깨 : 노무현. 보기 싫은듯요']


scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445754&bm=1
domain : 보배드림
title : 남양유업 보다 못한 0 0.
user_id : 나야나아재
작성 시간 :  2021.05.06
조회수 : 16
추천/반대 수 : 0 & 0
게시물 내용 : https://youtu.be/jFMPgGNDd_Q
댓글 수 : 0
게시물 

scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445741&bm=1
domain : 보배드림
title : 대깨문들은 외환보유액으로 문재인 찬양하는거 같단데.[6]
user_id : 뉘링부르2
작성 시간 :  2021.05.06
조회수 : 94
추천/반대 수 : 9 & 0
게시물 내용 : 올해 3월말 기준 우리나라의 외환보유액 규모는 사우디아라비아(4489억달러)에 추월 당하며 세계 9위로 다시 한 계단 내려갔다. 중국이 3조1700억달러로 부동의 1위를 지켰고 일본과 스위스가 각각 1조3685억달러와 1조520억달러로 2,3위를 차지했다.5770억달러를 보유한 인도는 러시아(5733억달러)를 제치고 4위에 올랐다. 대만은 5390억달러로 6위, 홍콩은 4914억달러로 7위 자리를 굳혔다.  그런데 왜 외환보유액으로 다변수 고려도 하지 않고 문재인 찬양하는거임????
댓글 수 : 6
게시물 댓글 : ['유교사상 : 본래 대깨문들은  내려갈때는 입닫고  올라간것만 문재인 업적으로들고오지요그러니  욕먹고  우리나라 발전의 암적인 존재입니다', '뉘링부르2 : 진짜 아무거나 오르면 찬양.. 내리면 입꾹닫.한미통화스와프는 mb때 해놨다던데요', '스피드가좋아 : 문재앙이 업적이 하도 자랑할께 없다보니 저런거라도 들고와야죠.', 'PAV : 나라가 망한다니까 하는 말이죠 ㅎㅎ 안망해요', '피톤치드 : 누렁아~ 요즘은 가출안하고 집 잘지키고 있냐~ 누렁아~ 수고만타~~~', '니궁디뒤에내가있다 : 대깨문들 숫자놀음에 선동 잘됩니다 뭐? 60배?']


scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445740&bm=1
domain : 보배드림
title : 문재인 종북 공산주의 독재자 천벌을 받으리라...2021.0

scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445732&bm=1
domain : 보배드림
title : 아직 출마선언도 안했는데, ㄷㄷㄷ[5]
user_id : 대가리깨진놈들
작성 시간 :  2021.05.06
조회수 : 103
추천/반대 수 : 5 & 0
게시물 내용 : 아직 출마 선언도 안했는데, 저정도면..ㄷㄷㄷ윤총장님 당선되면?? 정권교체 응원합니다.
댓글 수 : 5
게시물 댓글 : ['피톤치드 : 정치 1도 모르는 짜장이 찍어 주는 흑우는 베충이밖에 없을듯 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '시시오가이 : 누군 정치 잘 알아서 나라 꼴 이렇게 되게 만듬??ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '친일매국노일베충 : 제일 한심한 병신들이 검찰 빨아대는 극우꼴통들 ㅋ', '대가리깨진놈들 : 재앙이 빠는 깨문이도 있는데뭐..ㅎ', '시시오가이 : @대가리깨진놈들 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ깨문이는 사회악 ㅋㅋㅋㅋㅋㅋ']


scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445731&bm=1
domain : 보배드림
title : 왼쪽 조아라 하능 느그느그드를[5]
user_id : 무지개매너
작성 시간 :  2021.05.06
조회수 : 79
추천/반대 수 : 8 & 0
게시물 내용 : 가만 본깨     ㄱㅐ그즤 집단 맛당깨~
댓글 수 : 5
게시물 댓글 : ['대가리깨진놈들 : ㅋㅋㅋㅋ저런것도 좋다고 빨아주니,', '피톤치드 : 이 노땅새기 틀니좀 끼고 다녀라 제발 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '뉘링부르2 : 와.저거 사람이 할말인가?. 사회악 대깨문들은 조아라 하겠지??? 으휴', '이승만r : 세월호 쓰리썸 이 쌔꺄.', '발해대조영 : 와 시바 문씨인줄 알아따.느그느그느 다들 비슷

scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445746&bm=1
domain : 보배드림
title : 무주택 대깨문들 이제와서 양심 출타하심?[12]
user_id : 남고딩
작성 시간 :  2021.05.06
조회수 : 113
추천/반대 수 : 8 & 0
게시물 내용 : 미친새끼들 지들이 우리 이니 하고싶은거 다해~ 이지랄하면서 16년 17년에 전문가들이 이러면 공급막혀서 집값 폭등하고 결국 세입자들이 떠앉는다는 진단에도 투기꾼 일베충 취급하더니ㅋㅋㅋㅋㅋ결국 지들이 좆병신 벼락거지 됐네"아빠 우린 왜 또 이사가? 그냥 살면 안돼?" 너는 뉴스공장 들으면서 2시간 광역버스 출퇴근하면 되지만 갑자기 전학가는 자식새끼한테 미안하지도 않냐 ㅋㅋㅋ 그리고 너네는 문재인 찍은 댓가를 치루는거지만 문재인 안찍은 다음 세대한테 이 빅똥 어쩔거임?빨리 수도권을 떠나거나 일가족 번개탄이나 피워라
댓글 수 : 12
게시물 댓글 : ['피톤치드 : 동아일보 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 베충이들 교과서 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 셀프 인증요 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '친중매국노대깨 : 아따 베충이 반찬 없으면 밥도 못먹는 편식 깨문이 왔능가?', '피톤치드 : @친중매국노대깨 베충이를 왜 쳐먹냐 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 안팍으로 세균덩어리들인데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ베충이 새기 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '스피드가좋아 : 반론은 못하고 부들부들...겨우 한다는 게 조중동타령...측은하다', '피톤치드 : @스피드가좋아 베충이들 따위한테 뭔 반론이야 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 주제파악좀 해라 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '남고딩 : 대깨문 선생님 조선일보임다...', '친중매국노대깨 : ㅋㅋㅋㅋ 괜춘 어차피 40대 자식들은 피바다를 경험하게 된꺼임', '발해대조영 : 설마 보통 사람인줄 알앗겟지.이 정도로 박그네보다 못할줄이야 알앗겟냐.

scout : 2021.05.06 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=445735&bm=1
domain : 보배드림
title : 정동진 차이나타운 반대 국민청원 참여해주세요
user_id : 초딩곰
작성 시간 :  2021.05.06
조회수 : 27
추천/반대 수 : 2 & 0
게시물 내용 : 강원도지사 최문순이가 추진했다가 65만 국민 청원으로 재검토 처리된 차이나타운 기억하시죠. 이렇게 많은 국민이 반대하여 간신히 무산시켰는데 바로 옆 정동진에,"차이나드림타운" 아예 중국몽 타운을 건설한다고 지금 추진중입니다. 예전 차이나타운은 그나마 국내기업인 코오롱이 추진했다면,이번에는 아예 중국회사 중국자본이 국내에 들어와 추진하는 사업으로 이미 17만평 부지를 매입한 상태입니다. 국민의 불같은 반대로 무산되자 마자 바로 옆 동네에 이름만 바꾼 차이나타운을 건설하는 겁니다.이게 국민을 우롱하고 개돼지로 보는게 아니면 무엇일까요?좌우진영을 떠나 최문순이는 반드시 뒷조사를 해야 합니다 수상한점이 한둘이 아닙니다. 평소같아도 차이나타운 반대할텐데, 지금 중국은 김치, 한복부터 시작해서 동북공정 문화공정을 더 강하게 추진하고 있습니다.역사는 반복됩니다.. 저 오만한 짱개들은 대한민국을 독립된 국가로 인정하지도 존중하지도 않습니다.외교부장 왕이가 했던 망언들을 기억하세요, 야금야금 갉아먹으며 깊숙히 침투하는것이 저들의 전략입니다.10여년전 동북공정 얘기가 나올때 다들 대수롭지 않게 여겼지만 지금 어떻습니까..이미 짱개머니가 국내 언론, 방송가를 장악하고 있습니다. 조선구마사도 그 일부이구요. 더 늦기전에 국민들이 나서야 합니다. 중국은 한반도를 통째로 들어먹을 생각뿐입니다, 절대 우리를 이웃국가로 보지 않아요.제발 밑에 국민청원 참여해주시고 다른 곳에도 널리 알려주시길 부탁 드립니다. 이 문제는 좌우의 문제도, 세대의 문제도 남녀의 문제도 아닙니다.우리 한민족으로서 반드

...

scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442840&bm=1
domain : 보배드림
title : 교회 부활절 예배 끝난지 얼마 안되서 확진자 증가[2]
user_id : 닉네임없음3
작성 시간 :  2021.04.23
조회수 : 40
추천/반대 수 : 0 & 0
게시물 내용 : 그러나 정부는 교인에게 관대~~
댓글 수 : 2
게시물 댓글 : ['보으지헤프게써놓고 : 만히?', '엘리시아 : 교회에 하지말라고 했더니 정부가 종교 탄압 이라하더니...']


scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442838&bm=1
domain : 보배드림
title : 문재앙의 선거참패 보복[3]
user_id : 닉네임없음3
작성 시간 :  2021.04.23
조회수 : 99
추천/반대 수 : 12 & 0
게시물 내용 : 코로나 800명 넘어가도 그냥 냅두고 있음선거전엔 500명 넘어가도 2.5단계 가더니...이젠 그냥 선거에서 표 안줬으니 다 죽어라 마인드 선거에서 거의 몰빵 준 2030대 코린이들은성수 통해서  다 죽이고 있음... 문재앙 나이따 감옥 가는날 치킨 이벤트 합니다
댓글 수 : 3
게시물 댓글 : ['보으지헤프게써놓고 : 세후니가 열과 성을 다하는중 아닌가요?', '엘리시아 : 후니 이후부터 서울 확산세를 보거라', 'takeiteasy : 이렇게 일관성이 없어서 원.. 기준이란게 있긴 한건지ㅋㅋㅋㅋ진짜 아마추어 정부']


scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442837&bm=1
domain : 보배드림
title : 다 맞아가네요. ^&^[12]


scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442803&bm=1
domain : 보배드림
title : 보수 우파[21]
user_id : 발해대조영
작성 시간 :  2021.04.23
조회수 : 102
추천/반대 수 : 4 & 0
게시물 내용 : 선거에서 지 잘나서 이긴줄 알고 얼마됏다고 사면 얘기나 쳐하고 서병수는 탄핵이 잘못됏다하고 역사를 부정하는 이야길 하고잇 고,...그렇게 민주당이 쟈중 못해서 이 사단 난거 쟐 알면서  국힘당이 똑같은 실수를 반복하고 있다.구리고 기존 의원 정치인 들이 세력다툼 하는게 보인다.중진들은 물러나라.국힘이 살길 은 젊은 보수들이 이제ㅜ나서야한다.김세연을 필두로 젊은층이  앞장서야한다. 또 빨갱이들에게 빼앗길 것인가.
댓글 수 : 19
게시물 댓글 : ['가렴주구 : 요즘 국힘당도 헛소리 하는 인간들이 갑자기 늘어나고 있음. 이러니 국개라는 소리하지. 김일성 회고록 출판을 옹호하질 않나.', '발해대조영 : 진짭니까? 어찌 김일성회고록을 옹호를 하나..미친..누군데요? 그새끼가..', '발해대조영 : @가렴주구 />헉.하태경요? 우리 지역구인데..이샤끼 전향 안햇나?', '발해대조영 : @자칭보수것들아  태경이믄 간혹 한방씩 터트림.CIA 애서 정보를 빼내오는지 특종 한건씩 터트려서 사이다 핵 터트림.그래서 이새낄 놓질 못하겠슴.', 'roRnfTmdia : 국민의 힘당은 자신들을 지지해서 뽑아준게 아니란걸 알고 잘 대처해 나가야 할텐에요 ㄷ', '마취쟁이 : 지금 사면얘기 쳐 시불거리는것들과 분당하더라도 선을 그어야합니다. 도무지 대가리가 생각이 없어요.', '발해대조영 : 하..저도 미치갯내요.두번다시 이런 기회 잘 없을간데..ㅋㅋㅋ..젊은 보슈들은 안그러는데, 자리욕심잇는 새끼들이 극우짓을 한단 말입니다.요새 교활이도 술슬 태극기 시동 걸려하던데..하..젊운 보슈들이 전면에 나와야합니다'

scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442784&bm=1
domain : 보배드림
title : 테슬라가 천슬라 되는 방법^^[1]
user_id : 뉴복실은현다이
작성 시간 :  2021.04.23
조회수 : 65
추천/반대 수 : 0 & 0
게시물 내용 : 그거슨 ????  감자
댓글 수 : 1
게시물 댓글 : ['알긴뭘알어 : 고구마?? 촤하하하']


scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442780&bm=1
domain : 보배드림
title : 아침에 이거로 시작한다.[2]
user_id : 마취쟁이
작성 시간 :  2021.04.23
조회수 : 67
추천/반대 수 : 8 & 0
게시물 내용 : 문재인 양산에 오지말랜다.문재인 똥치우는 김부겸이랜다.
댓글 수 : 2
게시물 댓글 : ['조지큰루니 : 사저는 팽목항이 더 좋을것 같다는 개인생각 ㅋ', '스피드가좋아 : 재앙이 정신적 고향,전라도로 무조건 가야죠..임기내내 핵심요직에 전라도것들만 임명하고, 전라도 대통령노릇만 했으면 전라도로 가는 게 순리아님??그짝으로 가면 압도적지지 세력이 대환영할텐데 뭐가 걱정임??']


scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442779&bm=1
domain : 보배드림
title : 딸에 이어~[4]
user_id : 조지큰루니
작성 시간 :  2021.04.23
조회수 : 85
추천/반대 수 : 11 & 0
게시물 내용 : 아들마저 ㅋ
댓글 수 : 4
게시물 댓글 : ['뉘링부르2 : 당연히 취소해야죠.규정에도 있고.. 온라인 접수할때는 아예 첨부서류도 

scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442759&bm=1
domain : 보배드림
title : 씁쓸한 현실...[9]
user_id : 0뽀개드림0
작성 시간 :  2021.04.23
조회수 : 109
추천/반대 수 : 2 & 1
게시물 내용 : https://m.bobaedream.co.kr/board/bbs_view/best/416267/2/1
댓글 수 : 9
게시물 댓글 : ['대깨는질병 : 조선족...너 또 밤 샌건가? 한심한 놈', '알긴뭘알어 : 눈팅이라도 평소에 좀 하지 그르냐?글쓴이는 미쿡산다지금 미국은 몇시겠냐??', '당구장박사님짱 : @알긴뭘알어 대가리 깨져서ㅋㅋ그럼 낮에도 활개 치던데 그럼 그건???ㅋㅋ', '당구장박사님짱 : @알긴뭘알어 대가리 수준하고는 쯧쯧', '발해대조영 : 공산당 창당할라...냅둬라..유시티즌씨..', '알긴뭘알어 : 넌 공산당이 그렇게 좋냐?말만하면 빨갱이 빨갱이...가 북한가서 살아라~~!!^^', '당구장박사님짱 : @알긴뭘알어 저걸 공산당이 좋다고 보는 모지리ㅋㅋ에휴다마치기 어록 또 탄생ㅋ문재앙 비판기사면 임대 빠는거고유시민까면 공산단 빠는거고ㅋㅋ대가리 수준하고는 쯧쯧', '발해대조영 : @알긴뭘알어 ㅋㅋㅋ.피유웅쉰~', '마취쟁이 : 미국은? 공화당 또는 민주당인데일본은? 자민당 집권한지 수십년이고60 넘어서 뇌가 썩은 유시민은 믿고 걸러야지...유촉새는 통진당 당원들한테 빚진 8억 갚았나?']


scout : 2021.04.23 2021.04.23 True
target_url : https://www.bobaedream.co.kr/view?code=politic&No=442785&bm=1
domain : 보배드림
title : 이런갑질이  있나요 제보합니다
user_id : 복소리
작성 시간 :  2021.04.23
조회수 : 139
추천/반대 수 : 1 & 0


scout : 2021.04.22 2021.04.23 False
크롤링을 완료했습니다.


In [11]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
# df['keyword'] = pd.Series(keyword2)
df['title'] = pd.Series(title2)
df['user_name'] = pd.Series(user2)
df['url'] = pd.Series(url2)
df['date'] = pd.Series(time2)
df['view_cnt'] = pd.Series(view_cnt2)
df['like_cnt'] = pd.Series(like_cnt2)
df['user_text'] = pd.Series(text2)
df['com_cnt'] = pd.Series(com_cnt2)
df['com_text'] = pd.Series(com_text2)

df

,domain,title,writer,url,time,view_cnt,like_cnt,content,com_cnt,com_text
0,보배드림,"민변 “文정부, 민생 외면하고 檢개혁 몰두… 그마저도 낙제점”[2]",미국산앵무새,https://www.bobaedream.co.kr/view?code=politic...,2021.05.06,25,4 & 0,https://news.naver.com/main/read.nhn?mode=LSD&...,2,[모드칩 : 검찰 개혁이 결국 알고보니 퇴임 후 대통령의 안전확보였음을 국민이 알아...
1,보배드림,남양유업 보다 못한 0 0.,나야나아재,https://www.bobaedream.co.kr/view?code=politic...,2021.05.06,16,0 & 0,https://youtu.be/jFMPgGNDd_Q,0,[]
2,보배드림,강추하는 두 권의 책[2],국민윤리,https://www.bobaedream.co.kr/view?code=politic...,2021.05.06,31,0 & 0,"망월폐견'조국'과 '친일'의 두 가지 주제로 읽히더라구요.조국을 바라보는 언론, 검...",2,"[친중매국노대깨 : 지식의 암덩이를 추천하는건가, 국민윤리 : 재밋어 읽어봐]"
3,보배드림,허경영 욕한 대깨문색퀴들 다나와.,투태,https://www.bobaedream.co.kr/view?code=politic...,2021.05.06,57,5 & 0,기본소득 100만원군대전역 3000만원사회초년생 1억지금 민주당 씹새들이 하겠다는 ...,0,[]
4,보배드림,김어준은 성역이냐?[7],친중매국노대깨,https://www.bobaedream.co.kr/view?code=politic...,2021.05.06,87,7 & 0,신성불가침이랑께,7,"[더깨문 : 대깨들 이해가 안 감.저런 것들을 믿고 나불대는게., mosdef : ..."
...,...,...,...,...,...,...,...,...,...,...
1647,보배드림,보수참칭 수준,너의사랑에취하다,https://www.bobaedream.co.kr/view?code=politic...,2021.04.23,84,2 & 0,,0,[]
1648,보배드림,결국 백신 스와프도 실패하는건가?[2],방가농사꾼,https://www.bobaedream.co.kr/view?code=politic...,2021.04.23,161,5 & 0,백신추정주사라는 멍멍이소리 할때부터 불안했는데상황이 점점 안좋아지네요,2,"[colt69 : 미국이 우방.국민들땜에 체면치례상 쪼끔은 줄거 같음.ㅜㅜ, big..."
1649,보배드림,왜 기록을 포기하지[4],colt69,https://www.bobaedream.co.kr/view?code=politic...,2021.04.23,115,3 & 0,이해가 안되네?ㅋ,4,[국민웨건볼보 : 날짜 지났기 때문에 나와도 됩니다눈팅하고 있는 존배흥 나와라~!!...
1650,보배드림,결국 백신 스와프도 실패하는건가?[2],방가농사꾼,https://www.bobaedream.co.kr/view?code=politic...,2021.04.23,163,5 & 0,백신추정주사라는 멍멍이소리 할때부터 불안했는데상황이 점점 안좋아지네요,2,"[colt69 : 미국이 우방.국민들땜에 체면치례상 쪼끔은 줄거 같음.ㅜㅜ, big..."


In [13]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")



1.총 소요시간은 45125.8 초 입니다 
2.파일 저장 완료: csv 파일명 : C:\Users\JPARK-02\Desktop\crawler-bot\COMMUNITY\2021.04.23-2021.05.05\\bobe.csv 


SystemExit: 프로그램을 종료합니다.

c:\python39\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
